# Welcome to UBLyter !

_UBLyter_ is an [everis](https://www.everis.com/global/en) **SEMBU** environment for the generation of [OASIS UBL-2.x](http://docs.oasis-open.org/ubl/cs01-UBL-2.2/UBL-2.2.html) artefacts. The current version is UBLyter v0.1, which focuses only on the Pre-award stage. The status of the development is "just started!". Feel free to contribute by creating issues and commenting on the [Ublyter GitHub](https://github.com/everis-sembu/ublyter).

## Edit the model

* Download the [UBL-2.3 model](./mod/UBL-Pre-award-2.3.ods),

* Modify the model (introduce the changes you need),

* Upload it ([how to upload](http://everis-sembu.github.io/how-to-upload.html))

## Set the main distribution parameters

In [14]:
String 
    stage          = "preaward02", 
    version        = "test", // the name of the target resulting version
    datetimelocal  = "20190331-1500z"; // now


## Set the global variables

### Location of the `*.jar` files

Jupyter-notebook needs to know where to locate the saxon and other java resources. 

The code below is intended uniquely for Linux. It wont work for Windows. Check this URL for an example of how to handle this in other OS: https://stackoverflow.com/questions/318239/how-do-i-set-environment-variables-from-java

#### Set environment variables

In [22]:
import java.lang.reflect.Field;

try {
        Map<String, String> env = System.getenv();
        Class<?> cl = env.getClass();
        Field field = cl.getDeclaredField("m");
        field.setAccessible(true);
        Map<String, String> writableEnv = (Map<String, String>) field.get(env);
        writableEnv.put("IJAVA_CLASSPATH", "./lib/*.jar");
    } catch (Exception e) {
        throw new IllegalStateException("Failed to set environment variable", e);
    }



#### Check that the environment variables are seen by the Jupyter-notebook

In [21]:
Map<String, String> env = System.getenv();
String value = env.get( "IJAVA_CLASSPATH" );
System.out.println(value);

./lib/*.jar
